In [ ]:
import unittest
from unittest.mock import patch, Mock
from jobs.bronze_layer import BronzeLayerProcessor, APIClient
from plugins.storage_client import AzureStorageClient


class TestBronzeLayerProcessor(unittest.TestCase):

    @patch('jobs.bronze_layer.AzureStorageClient')
    @patch('airflow.providers.http.operators.http.HttpOperator')
    def test_bronze_layer_successful(self, MockHttpOperator, MockAzureStorageClient):
        """Test BronzeLayerProcessor with a successful API response"""

        mock_http_operator_instance = MockHttpOperator.return_value
        mock_http_operator_instance.execute.return_value = '{"breweries": [{"id": 1, "name": "Brewery"}]}'

        mock_storage_client_instance = MockAzureStorageClient.return_value

        api_client = APIClient(
            http_conn_id='breweries_api_connection',
            endpoint='breweries'
        )
        storage_client = mock_storage_client_instance

        processor = BronzeLayerProcessor(
            api_client=api_client, storage_client=storage_client)

        processor.process()

        MockHttpOperator.assert_called_once_with(
            task_id='get_api_data',
            http_conn_id='breweries_api_connection',
            endpoint='breweries',
            method='GET',
            headers={"Content-Type": "application/json"}
        )

        mock_storage_client_instance.upload_blob.assert_called_once_with(
            blob_name='breweries_data.json',
            container='bronze-layer',
            data='{"breweries": [{"id": 1, "name": "Brewery"}]}'
        )

    @patch('jobs.bronze_layer.AzureStorageClient')
    @patch('airflow.providers.http.operators.http.HttpOperator')
    def test_bronze_layer_api_failure(self, MockHttpOperator, MockAzureStorageClient):
        """Test BronzeLayerProcessor when the API call fails"""

        mock_http_operator_instance = MockHttpOperator.return_value
        mock_http_operator_instance.execute.side_effect = Exception(
            "API call failed")

        mock_storage_client_instance = MockAzureStorageClient.return_value

        api_client = APIClient(
            http_conn_id='breweries_api_connection',
            endpoint='breweries'
        )
        storage_client = mock_storage_client_instance

        processor = BronzeLayerProcessor(
            api_client=api_client, storage_client=storage_client)

        with self.assertRaises(Exception) as context:
            processor.process()

        self.assertEqual(str(context.exception), "API call failed")

        mock_storage_client_instance.upload_blob.assert_not_called()


if __name__ == '__main__':
    unittest.main()